# Creating Features Quiz
Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [4]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) run the cells below to read dataset and build body length feature
# 3) write code to answer the quiz questions 
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, \
    IDF, StringIndexer, VectorAssembler, Normalizer, StandardScaler, MinMaxScaler
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import concat
from pyspark.sql import functions as f

import re

In [5]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

### Read Dataset

In [6]:
stack_overflow_data = 'Train_onetag_small.json'

In [7]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

### Build Body Length Feature

In [8]:
regexTokenizer = RegexTokenizer(inputCol="Body", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)

In [9]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn("BodyLength", body_length(df.words))

In [10]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

# Question 1
Select the question with Id = 1112. How many words does its body contain (check the BodyLength column)?

In [11]:
# TODO: write your code to answer question 1
df.select('BodyLength').where(df.Id == '5123').show()

+----------+
|BodyLength|
+----------+
|       132|
+----------+



# Question 2
Create a new column that concatenates the question title and body. Apply the same functions we used before to compute the number of words in this combined column. What's the value in this new column for Id = 5123?

In [12]:
# TODO: write your code to answer question 2
df_1 = df.withColumn('body_title',concat(f.col('Body'), f.lit('_'), f.col('Title')))
df_1 = df_1.drop('Body', 'Title', 'words')
regexTokenizer = RegexTokenizer(inputCol="body_title", outputCol="words1", pattern="\\W")
df_1= regexTokenizer.transform(df_1)
df_1= df_1.withColumn("BodyTitleLength", body_length(df_1.words1))
df_1.select('BodyTitleLength').where(df_1.Id == '5123').show()

+---------------+
|BodyTitleLength|
+---------------+
|            135|
+---------------+



# Create a Vector
Create a vector from the combined Title + Body length column. In the next few questions, you'll try different normalizer/scaler methods on this new column.

In [13]:
# TODO: write your code to create this vector
assembler = VectorAssembler(inputCols=["BodyTitleLength"], outputCol="NumFeatures")
df_1 = assembler.transform(df_1)

# Question 3
Using the Normalizer method what's the normalized value for question Id = 512?

In [14]:
# TODO: write your code to answer question 3
scaler = Normalizer(inputCol = "NumFeatures", outputCol = "ScalNumFeatures_1")
df_1 = scaler.transform(df_1)
df_1.select('ScalNumFeatures_1').where(df_1.Id == 512).show()

+-----------------+
|ScalNumFeatures_1|
+-----------------+
|            [1.0]|
+-----------------+



# Question 4
Using the StandardScaler method (scaling both the mean and the standard deviation) what's the normalized value for question Id = 512?

In [15]:
# TODO: write your code to answer question 4
scaler2 = StandardScaler(inputCol="NumFeatures", outputCol="ScaledNumFeatures_3", withMean = True, withStd=True)
scalerModel = scaler2.fit(df_1)
df_1 = scalerModel.transform(df_1)
df_1.select('ScaledNumFeatures_3').where(df_1.Id == 512).show()

+--------------------+
| ScaledNumFeatures_3|
+--------------------+
|[-0.6417775027593...|
+--------------------+



# Question 5
Using the MinMAxScaler method what's the normalized value for question Id = 512?

In [18]:
# TODO: write your code to answer question 5
scaler3 = MinMaxScaler(inputCol="NumFeatures", outputCol="ScaledNumFeatures_MM")
MinMaxscalerModel = scaler3.fit(df_1)
df_1 = MinMaxscalerModel.transform(df_1)
df_1.select('ScaledNumFeatures_MM').where(df_1.Id == 512).show()

+--------------------+
|ScaledNumFeatures_MM|
+--------------------+
|[0.00624833820792...|
+--------------------+

